In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:

import os
import torch
from PIL import Image
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import AutoProcessor, AutoModelForZeroShotObjectDetection
import pandas as pd

In [ ]:
#src = str(Path.cwd().resolve().parents[1]/ 'src')
#sys.path.append(src)
#from config.paths import ROOT, DATA, EXPERIMENTS, DOC_IMGS, CLEANED, PROCESSED, METADATA

In [3]:
EXPERIMENTS = "/content/drive/MyDrive/dataHN/experiments"
DATA = "/content/drive/MyDrive/dataHN/data"

In [ ]:

model_id = "IDEA-Research/grounding-dino-tiny"
device = "cuda" if torch.cuda.is_available() else "cpu"
size = 'tiny'

processor = AutoProcessor.from_pretrained(model_id)
model = AutoModelForZeroShotObjectDetection.from_pretrained(model_id).to(device)



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/457 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/689M [00:00<?, ?B/s]

# Objets

In [5]:
obj_dir = '/content/drive/MyDrive/dataHN/data/cleaned/obj_imgs_cleaned'
len(os.listdir(obj_dir))

1758

In [ ]:
text_labels = [["object"]]

In [ ]:

dino_dir = os.path.join(EXPERIMENTS, 'dino_obj_detect')
results_dir = os.path.join(dino_dir, f'results_{size}')
os.makedirs(results_dir, exist_ok=True)
image_dir = os.path.join(DATA, 'cleaned', 'obj_imgs_cleaned')
image_files = [f for f in os.listdir(image_dir) if f.lower().endswith((".jpg", ".jpeg", ".png"))]

csv_path = os.path.join(results_dir, "detections.csv")
df = pd.DataFrame(columns=["image_file", "label", "score", "x_min", "y_min", "x_max", "y_max"])

In [ ]:
model.eval()

for filename in tqdm(image_files, desc='Processing image'):
    path = os.path.join(image_dir, filename)
    try:
        image = Image.open(path).convert("RGB")
    except Exception as e:
        print(f"Erreur : {path} (error: {e})")
        continue

    inputs = processor(images=image, text=text_labels, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model(**inputs)

    results = processor.post_process_grounded_object_detection(
        outputs,
        threshold=0.4,
        text_threshold=0.3,
        target_sizes=[(image.height, image.width)]
    )

    result = results[0]
    for box, score, text_label in zip(result["boxes"], result["scores"], result["text_labels"]):
        box = [round(x, 2) for x in box.tolist()]
        df.loc[len(df)] = [filename, text_label, round(score.item(), 3)] + box



Processing image: 100%|██████████| 1757/1757 [15:38<00:00,  1.87it/s]


In [ ]:
df.to_csv(csv_path, index=False)
print(f"Détection terminée. Résultats enregistrés dans : {csv_path}")

Détection terminée. Résultats enregistrés dans : /content/drive/MyDrive/dataHN/experiments/dino_obj_detect/results_tiny/detections.csv


------

## Visualization

In [ ]:
import os
import pandas as pd
from PIL import Image, ImageDraw

# Your resizing function (adapted to take a PIL image)
def resize_save_img_from_PIL(img, output_path, target_size=640):
    w, h = img.size
    if w > h:
        new_w = target_size
        new_h = int(h * target_size / w)
    else:
        new_h = target_size
        new_w = int(w * target_size / h)
    img_resized = img.resize((new_w, new_h), Image.BILINEAR)
    img_resized.save(output_path, quality=90)



In [ ]:
# csv_path = os.path.join(results_dir, "detections.csv")
output_dir = os.path.join(results_dir, "annotated_resized")
os.makedirs(output_dir, exist_ok=True)

#df = pd.read_csv(csv_path)

grouped = df.groupby("image_file")

In [ ]:
from PIL import ImageDraw, ImageFont

for img_file, group in grouped:
    img_path = os.path.join(image_dir, img_file)

    try:
        img = Image.open(img_path).convert("RGB")
    except Exception as e:
        print(f"Erreur : {img_path} (error: {e})")
        continue

    draw = ImageDraw.Draw(img)
    try:
        font = ImageFont.truetype("arial.ttf", size=16)
    except:
        font = ImageFont.load_default()

    for _, row in group.iterrows():
        box = [row["x_min"], row["y_min"], row["x_max"], row["y_max"]]
        label = f'{row["label"]} {row["score"]:.2f}'

        # bbox
        draw.rectangle(box, outline="red", width=5)

        # label background using textbbox (Pillow ≥ 8.0)
        text_bbox = draw.textbbox((box[0], box[1]), label, font=font)
        text_w = text_bbox[2] - text_bbox[0]
        text_h = text_bbox[3] - text_bbox[1]

        draw.rectangle(
            [box[0], box[1] - text_h, box[0] + text_w, box[1]],
            fill="red"
        )

        # label text
        draw.text((box[0], box[1] - text_h), label, fill="white", font=font)

    # Save image
    output_path = os.path.join(output_dir, os.path.basename(img_path))
    resize_save_img_from_PIL(img, output_path)

print(f"Images enregistrées dans : {output_dir}")

Images enregistrées dans : /content/drive/MyDrive/dataHN/experiments/dino_obj_detect/results_tiny/annotated_resized


# Illustrations

In [ ]:
text_labels = [["illustration"]]

In [ ]:

dino_dir = os.path.join(EXPERIMENTS, 'dino_ill_detect')
results_dir = os.path.join(dino_dir, f'results_{size}')
os.makedirs(results_dir, exist_ok=True)
image_dir = os.path.join(DATA, 'cleaned', 'doc_imgs_cleaned')
image_files = [f for f in os.listdir(image_dir) if f.lower().endswith((".jpg", ".jpeg", ".png"))]

csv_path = os.path.join(results_dir, "detections.csv")
df = pd.DataFrame(columns=["image_file", "label", "score", "x_min", "y_min", "x_max", "y_max"])

In [ ]:
model.eval()

for filename in tqdm(image_files, desc='Processing image'):
    path = os.path.join(image_dir, filename)
    try:
        image = Image.open(path).convert("RGB")
    except Exception as e:
        print(f"Erreur : {path} (error: {e})")
        continue

    #  inputs
    inputs = processor(images=image, text=text_labels, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model(**inputs)

    results = processor.post_process_grounded_object_detection(
        outputs,
        threshold=0.4,
        text_threshold=0.3,
        target_sizes=[(image.height, image.width)]
    )

    # to DataFrame
    result = results[0]
    for box, score, text_label in zip(result["boxes"], result["scores"], result["text_labels"]):
        box = [round(x, 2) for x in box.tolist()]
        df.loc[len(df)] = [filename, text_label, round(score.item(), 3)] + box



Processing image:  64%|██████▍   | 1244/1932 [27:53<30:21,  2.65s/it]/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3452: DecompressionBombWarning: Image size (131542390 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
Processing image:  65%|██████▍   | 1250/1932 [28:11<33:00,  2.90s/it]/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3452: DecompressionBombWarning: Image size (131396860 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
Processing image: 100%|██████████| 1932/1932 [44:20<00:00,  1.38s/it]


In [ ]:
# Save
df.to_csv(csv_path, index=False)
print(f"Détection terminée. Résultats enregistrés dans : {csv_path}")

Détection terminée. Résultats enregistrés dans : /content/drive/MyDrive/dataHN/experiments/dino_ill_detect/results_tiny/detections.csv


## Visualization

In [ ]:
df = pd.read_csv(csv_path)

In [ ]:
import os
import pandas as pd
from PIL import Image, ImageDraw

# Your resizing function (adapted to take a PIL image)
def resize_save_img_from_PIL(img, output_path, target_size=640):
    w, h = img.size
    if w > h:
        new_w = target_size
        new_h = int(h * target_size / w)
    else:
        new_h = target_size
        new_w = int(w * target_size / h)
    img_resized = img.resize((new_w, new_h), Image.BILINEAR)
    img_resized.save(output_path, quality=90)



In [ ]:
# Paths
# csv_path = os.path.join(results_dir, "detections.csv")
output_dir = os.path.join(results_dir, "annotated_resized")
os.makedirs(output_dir, exist_ok=True)

#df = pd.read_csv(csv_path)

grouped = df.groupby("image_file")

In [ ]:
from PIL import ImageDraw, ImageFont

for img_file, group in grouped:
    img_path = os.path.join(image_dir, img_file)

    try:
        img = Image.open(img_path).convert("RGB")
    except Exception as e:
        print(f"Erreur : {img_path} (error: {e})")
        continue

    draw = ImageDraw.Draw(img)
    try:
        font = ImageFont.truetype("arial.ttf", size=16)
    except:
        font = ImageFont.load_default()

    for _, row in group.iterrows():
        box = [row["x_min"], row["y_min"], row["x_max"], row["y_max"]]
        label = f'{row["label"]} {row["score"]:.2f}'

        # bbox
        draw.rectangle(box, outline="red", width=5)

        # label background using textbbox (Pillow ≥ 8.0)
        text_bbox = draw.textbbox((box[0], box[1]), label, font=font)
        text_w = text_bbox[2] - text_bbox[0]
        text_h = text_bbox[3] - text_bbox[1]

        draw.rectangle(
            [box[0], box[1] - text_h, box[0] + text_w, box[1]],
            fill="red"
        )

        # label text
        draw.text((box[0], box[1] - text_h), label, fill="white", font=font)

    # Save image
    output_path = os.path.join(output_dir, os.path.basename(img_path))
    resize_save_img_from_PIL(img, output_path)

print(f"Images enregistrées dans : {output_dir}")

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3452: DecompressionBombWarning: Image size (131542390 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3452: DecompressionBombWarning: Image size (131396860 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Images enregistrées dans : /content/drive/MyDrive/dataHN/experiments/dino_ill_detect/results_tiny/annotated_resized


### Ajout detection

In [19]:
df_detections = pd.read_csv(csv_path)

In [20]:
df_detections.columns

Index(['image_file', 'label', 'score', 'x_min', 'y_min', 'x_max', 'y_max'], dtype='object')

In [21]:
df_detections = pd.read_csv(csv_path)
print(len(df_detections.groupby("image_file")))
print(len(os.listdir(obj_dir)))

1756
1758


In [22]:
missing_filenames = [f for f in os.listdir(obj_dir) if f not in df_detections["image_file"].unique()]
missing_filenames

['MTD_THD-DB-2002--1--95_03.jpg', 'Tajan_2012-05-24_213.jpg']

In [23]:
df_new = df_detections.copy()

In [24]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

for filename in tqdm(missing_filenames, desc='Processing image'):
    path = os.path.join(obj_dir, filename)
    try:
        image = Image.open(path).convert("RGB")
    except Exception as e:
        print(f"Erreur : {path} (error: {e})")
        continue

    # inputs
    inputs = processor(images=image, text=text_labels, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model(**inputs)

    results = processor.post_process_grounded_object_detection(
        outputs,
        threshold=0.4,
        text_threshold=0.3,
        target_sizes=[(image.height, image.width)]
    )

    # Visualisation des images manquantes avec bboxes
    result = results[0]

    fig, ax = plt.subplots(1, figsize=(10, 10))
    ax.imshow(image)

    for box, score, text_label in zip(result["boxes"], result["scores"], result["text_labels"]):
        box = [round(x, 2) for x in box.tolist()]
        xmin, ymin, xmax, ymax = box

        # draw rectangle
        rect = patches.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                 linewidth=2, edgecolor="red", facecolor="none")
        ax.add_patch(rect)

        # add label + score
        ax.text(xmin, ymin - 5, f"{text_label} {score:.2f}",
                color="red", fontsize=10, backgroundcolor="white")

    plt.axis("off")
    plt.show()

    # to DataFrame
    for box, score, text_label in zip(result["boxes"], result["scores"], result["text_labels"]):
        box = [round(x, 2) for x in box.tolist()]
        df_new.loc[len(df_new)] = [filename, text_label, round(score.item(), 3)] + box

Output hidden; open in https://colab.research.google.com to view.

In [25]:
df_detections.tail(10)

,image_file,label,score,x_min,y_min,x_max,y_max
2116,MTD_THD-993--3--10_01.jpg,object,0.781,59.94,65.29,1142.68,1147.27
2117,"SMBKM_1867,173_01.jpg",object,0.750,1304.29,318.65,2018.55,1916.36
2118,"SMBKM_1867,173_01.jpg",object,0.738,295.39,310.82,1009.15,1913.30
2119,MTD_THD-DB-2002--1--95_04.jpg,object,0.797,74.83,78.95,723.97,1196.72
2120,VA_8098-1863_01.jpg,object,0.725,54.99,43.31,260.16,512.14
2121,VA_8098-1863_01.jpg,object,0.694,435.17,58.13,633.41,506.81
2122,VA_8098-1863_01.jpg,object,0.660,268.88,201.69,422.81,529.68
2123,CB_GF4232_01.jpg,object,0.687,53.75,122.03,546.74,617.81
2124,MMNS_MNC6654-B_01.jpg,object,0.877,98.31,141.03,594.30,479.42
2125,VA_O150508_01.jpg,object,0.660,33.11,352.35,1798.04,2125.45


In [26]:
df_new.tail(10)

,image_file,label,score,x_min,y_min,x_max,y_max
2117,"SMBKM_1867,173_01.jpg",object,0.750,1304.29,318.65,2018.55,1916.36
2118,"SMBKM_1867,173_01.jpg",object,0.738,295.39,310.82,1009.15,1913.30
2119,MTD_THD-DB-2002--1--95_04.jpg,object,0.797,74.83,78.95,723.97,1196.72
2120,VA_8098-1863_01.jpg,object,0.725,54.99,43.31,260.16,512.14
2121,VA_8098-1863_01.jpg,object,0.694,435.17,58.13,633.41,506.81
2122,VA_8098-1863_01.jpg,object,0.660,268.88,201.69,422.81,529.68
2123,CB_GF4232_01.jpg,object,0.687,53.75,122.03,546.74,617.81
2124,MMNS_MNC6654-B_01.jpg,object,0.877,98.31,141.03,594.30,479.42
2125,VA_O150508_01.jpg,object,0.660,33.11,352.35,1798.04,2125.45
2126,Tajan_2012-05-24_213.jpg,object,0.829,24.72,17.37,585.90,517.80


In [27]:
df_new.to_csv(csv_path, index=False)
